In [1]:
import torch
import numpy as np
import pandas as pd
pd.set_option("display.width", 380)
pd.set_option('max_colwidth', 100)

from IPython.display import display

device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')

# Load the sentences

In [2]:
df_train = pd.read_csv("./sick_train/SICK_train.txt", sep="\t")
df_train = df_train.drop(['relatedness_score'], axis=1)

df_test = pd.read_csv("./sick_test/SICK_test.txt", sep="\t")
df_test = df_test.drop(['relatedness_score'], axis=1)

In [3]:
from torch.utils.data import Dataset
from torch.utils.data.dataloader import DataLoader

In [4]:
from gensim import corpora

In [5]:
class SickDataset(Dataset):
    endOfSentence   = '</s>'
    startOfSentence = '<s>'
    separator2Sentences = '<sep>'
    
    tokens = [startOfSentence, separator2Sentences, endOfSentence]
    
    def join_sentence(self, row):
        """
        Create a new sentence (<s> + s_A + <sep> + s_B + </s>)
        """
        sentence_a = row['sentence_A'].split(" ")
        sentence_b = row['sentence_B'].split(" ")
        return np.concatenate((
            [self.startOfSentence],
            sentence_a,
            [self.separator2Sentences],
            sentence_b,
            [self.endOfSentence]
        ))
    
    def series_text_2_labelID(self, series, keep_n=1000):
        """
        Convert text Label into label id
        """
        return series.map({"NEUTRAL": 0, "ENTAILMENT": 1, "CONTRADICTION": 2})
    
    def series_2_dict(self, series, keep_n):
        """
        Convert document (a list of words) into a list of indexes
        AND apply some filter on the documents
        """
        dictionary = corpora.Dictionary(series)
        dictionary.filter_extremes(
            no_below=1,
            no_above=1,
            keep_n=keep_n,
            keep_tokens=self.tokens)
        return dictionary
    
    
    def __init__(self, df, vocabulary_size, dic = None):
        # Label text as ids
        df["entailment_id"] = self.series_text_2_labelID(df['entailment_judgment'])
        
        # Add <s>,</s>,<sep> tokens to the vocabulary
        df['sentence_AB'] = df.apply(self.join_sentence, axis=1)
        
        # check if the dictionary is given
        if dic is None:
            # Create the Dictionary
            self.dictionary = self.series_2_dict(df['sentence_AB'], vocabulary_size)
        else:
            self.dictionary = dic
        
        # sentence of words -> array of idx
        # Adds unknown to the voc (idx = vocabulary_size), Dictionary size = vocabulary_size+1
        df["word_idx"] = df["sentence_AB"].apply(
            lambda x: np.array(self.dictionary.doc2idx(x, unknown_word_index=vocabulary_size))
        )
        
        self.df = df
        
    def getRef(self, index):
        return self.df['sentence_AB'][index]
        
    def __getitem__(self, index):
        return (
            self.df['word_idx'][index],
            self.df['entailment_id'][index])
    
    def getDictionary(self):
        return self.dictionary
    
    def __len__(self):
        return len(self.df)

vocabulary_size = 1000
sick_dataset_train = SickDataset(df_train, vocabulary_size)
sick_dataset_train.df.head()

,pair_ID,sentence_A,sentence_B,entailment_judgment,entailment_id,sentence_AB,word_idx
0,1,A group of kids is playing in a yard and an old man is standing in the background,A group of boys in a yard is playing and a man is standing in the background,NEUTRAL,0,"[<s>, A, group, of, kids, is, playing, in, a, yard, and, an, old, man, is, standing, in, the, ba...","[1, 3, 9, 14, 12, 11, 16, 10, 4, 19, 6, 5, 15, 13, 11, 17, 10, 18, 7, 2, 3, 9, 14, 8, 10, 4, 19,..."
1,2,A group of children is playing in the house and there is no man standing in the background,A group of kids is playing in a yard and an old man is standing in the background,NEUTRAL,0,"[<s>, A, group, of, children, is, playing, in, the, house, and, there, is, no, man, standing, in...","[1, 3, 9, 14, 20, 11, 16, 10, 18, 21, 6, 23, 11, 22, 13, 17, 10, 18, 7, 2, 3, 9, 14, 12, 11, 16,..."
2,3,The young boys are playing outdoors and the man is smiling nearby,The kids are playing outdoors near a man with a smile,ENTAILMENT,1,"[<s>, The, young, boys, are, playing, outdoors, and, the, man, is, smiling, nearby, <sep>, The, ...","[1, 24, 31, 8, 25, 16, 27, 6, 18, 13, 11, 29, 1000, 2, 24, 12, 25, 16, 27, 26, 4, 13, 30, 4, 28, 0]"
3,5,The kids are playing outdoors near a man with a smile,A group of kids is playing in a yard and an old man is standing in the background,NEUTRAL,0,"[<s>, The, kids, are, playing, outdoors, near, a, man, with, a, smile, <sep>, A, group, of, kids...","[1, 24, 12, 25, 16, 27, 26, 4, 13, 30, 4, 28, 2, 3, 9, 14, 12, 11, 16, 10, 4, 19, 6, 5, 15, 13, ..."
4,9,The young boys are playing outdoors and the man is smiling nearby,A group of kids is playing in a yard and an old man is standing in the background,NEUTRAL,0,"[<s>, The, young, boys, are, playing, outdoors, and, the, man, is, smiling, nearby, <sep>, A, gr...","[1, 24, 31, 8, 25, 16, 27, 6, 18, 13, 11, 29, 1000, 2, 3, 9, 14, 12, 11, 16, 10, 4, 19, 6, 5, 15..."


In [6]:
dictionary_train = sick_dataset_train.getDictionary()
print(dictionary_train)
vocabulary_size = len(dictionary_train)

sick_dataset_test = SickDataset(df_test, vocabulary_size, dictionary_train)
dictionary_test = sick_dataset_test.getDictionary()
print(dictionary_test)
sick_dataset_test.df.head()

Dictionary(1000 unique tokens: ['', 'begging', 'thin', 'mixture', 'rising']...)
Dictionary(1000 unique tokens: ['', 'begging', 'thin', 'mixture', 'rising']...)


,pair_ID,sentence_A,sentence_B,entailment_judgment,entailment_id,sentence_AB,word_idx
0,6,There is no boy playing outdoors and there is no man smiling,A group of kids is playing in a yard and an old man is standing in the background,NEUTRAL,0,"[<s>, There, is, no, boy, playing, outdoors, and, there, is, no, man, smiling, <sep>, A, group, ...","[1, 100, 11, 22, 148, 16, 27, 6, 23, 11, 22, 13, 29, 2, 3, 9, 14, 12, 11, 16, 10, 4, 19, 6, 5, 1..."
1,7,A group of boys in a yard is playing and a man is standing in the background,The young boys are playing outdoors and the man is smiling nearby,NEUTRAL,0,"[<s>, A, group, of, boys, in, a, yard, is, playing, and, a, man, is, standing, in, the, backgrou...","[1, 3, 9, 14, 8, 10, 4, 19, 11, 16, 6, 4, 13, 11, 17, 10, 18, 7, 2, 24, 31, 8, 25, 16, 27, 6, 18..."
2,8,A group of children is playing in the house and there is no man standing in the background,The young boys are playing outdoors and the man is smiling nearby,NEUTRAL,0,"[<s>, A, group, of, children, is, playing, in, the, house, and, there, is, no, man, standing, in...","[1, 3, 9, 14, 20, 11, 16, 10, 18, 21, 6, 23, 11, 22, 13, 17, 10, 18, 7, 2, 24, 31, 8, 25, 16, 27..."
3,10,A brown dog is attacking another animal in front of the tall man in pants,A brown dog is attacking another animal in front of the man in pants,ENTAILMENT,1,"[<s>, A, brown, dog, is, attacking, another, animal, in, front, of, the, tall, man, in, pants, <...","[1, 3, 38, 39, 11, 1000, 37, 36, 10, 40, 14, 18, 268, 13, 10, 41, 2, 3, 38, 39, 11, 1000, 37, 36..."
4,11,A brown dog is attacking another animal in front of the man in pants,A brown dog is helping another animal in front of the man in pants,NEUTRAL,0,"[<s>, A, brown, dog, is, attacking, another, animal, in, front, of, the, man, in, pants, <sep>, ...","[1, 3, 38, 39, 11, 1000, 37, 36, 10, 40, 14, 18, 13, 10, 41, 2, 3, 38, 39, 11, 1000, 37, 36, 10,..."


In [7]:
pd.DataFrame(list(zip(sick_dataset_train.getRef(2), sick_dataset_train[2][0]))).T

,0,1,2,3,4,5,6,7,8,9,...,16,17,18,19,20,21,22,23,24,25
0,<s>,The,young,boys,are,playing,outdoors,and,the,man,...,are,playing,outdoors,near,a,man,with,a,smile,</s>
1,1,24,31,8,25,16,27,6,18,13,...,25,16,27,26,4,13,30,4,28,0


# Create the RNN Model

In [8]:
from torch import nn

In [9]:
class RNNClassifier(nn.Module):
    # Our model

    def __init__(self, input_voc_size, embedding_size, hidden_size):
        super(RNNClassifier, self).__init__()
        
        self.input_voc_size = input_voc_size
        self.embedding_size = embedding_size
        self.hidden_size = hidden_size
        
        self.num_classes = 3
        
        # Add the unknown token
        self.embedding = nn.Embedding(input_voc_size+1, embedding_size)
        self.rnn = nn.RNN(
              input_size=embedding_size,
              hidden_size=hidden_size,
              batch_first=True)
        self.fc = nn.Linear(hidden_size, self.num_classes)
        
    def forward(self, x, verbose=False):
        # Initialize hidden and cell states
        # (num_layers * num_directions, batch, hidden_size)
        h_0 = torch.zeros(1, 1, self.hidden_size).to(device)
        # When creating new variables inside a model (like the hidden state in an RNN/GRU/LSTM),
        # make sure to also move them to the device (GPU).

        if verbose:
            print("  input", x.size())
        emb = self.embedding(x)
        emb = emb.view(1, x.size(0), -1)
        if verbose:
            print("  embedding", emb.size())

        # Propagate embedding through RNN
        # Input: (batch, seq_len, embedding_size)
        # h_0: (num_layers * num_directions, batch, hidden_size)
        out, hidden = self.rnn(emb, h_0)
        if verbose:
            print("  rnn_out", out.size())
        return self.fc(hidden)
    
rnn = RNNClassifier(vocabulary_size, 100, 10)
rnn.to(device)
print(rnn)

# Set loss and optimizer function
# CrossEntropyLoss = LogSoftmax + NLLLoss
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(rnn.parameters(), lr=0.01)

RNNClassifier(
  (embedding): Embedding(1001, 100)
  (rnn): RNN(100, 10, batch_first=True)
  (fc): Linear(in_features=10, out_features=3, bias=True)
)


# To DataLoader

In [10]:
from torch.utils.data import DataLoader

In [11]:
# TODO, use this insted
train_loader = DataLoader(dataset=sick_dataset_train,
                          batch_size=1, shuffle=True)

test_loader = DataLoader(dataset=sick_dataset_test,
                          batch_size=1, shuffle=True)

# Training the Model 

In [12]:
%%time
# Train the model
verbose = False

num_epochs=2

rnn.train()
rnn_losses = []

for epoch in range(num_epochs):
    total_correct = 0
    total_target = 0
    for batch_idx, (data, target) in enumerate(train_loader):
        
        data = data.to(device)
        target = target.to(device)
        
        idx_sentences = data.view(-1)
        if verbose:
            print('idx_sentences', idx_sentences.size())
                
        target = target
        if verbose:
            print('target', target)
        
        output = rnn(idx_sentences)[0]
        if verbose:
            print('output', output)
        
        loss = criterion(output, target)
        if verbose:
            print('lose', loss)
        
        rnn.zero_grad()
        loss.backward()
        optimizer.step()
        
        # Get the Accuracy
        _, predicted = torch.max(output.data, dim=1)
        correct = (predicted == target).sum().item()
        
        total_correct += correct
        total_target += target.size(0)
        
        if batch_idx % 100 == 0 or batch_idx % 100 == 1 :
            print('\r Train Epoch: [{}/{}] | Step [{:5}/{} ({:.0f}%)] | Loss: {:.4f} | Accuracy: {:.4f}%'.format(
                    epoch+1, num_epochs,
                    batch_idx * len(data), 
                    len(train_loader.dataset),
                    100. * batch_idx / len(train_loader), 
                    loss.item(), 
                    (total_correct / total_target) * 100), end=' ')
            
    print()


print("Learning finished!")

 Train Epoch: [1/2] | Step [ 4401/4500 (98%)] | Loss: 1.7585 | Accuracy: 54.3389% 
 Train Epoch: [2/2] | Step [ 4401/4500 (98%)] | Loss: 0.6678 | Accuracy: 53.8392% 
Learning finished!
CPU times: user 35.8 s, sys: 1.47 s, total: 37.3 s
Wall time: 37 s


# Evaluate the model

In [15]:
%%time

total_correct = 0
total_target = 0
for batch_idx, (data, target) in enumerate(test_loader):
        
        data = data.to(device)
        target = target.to(device)
        
        idx_sentences = data.view(-1)
        if verbose:
            print('idx_sentences', idx_sentences.size())
                
        target = target
        if verbose:
            print('target', target)
        
        output = rnn(idx_sentences)[0]
        if verbose:
            print('output', output)
        
        # Get the Accuracy
        _, predicted = torch.max(output.data, dim=1)
        correct = (predicted == target).sum().item()
        
        total_correct += correct
        total_target += target.size(0)
        
        if batch_idx % 100 == 0 or batch_idx % 100 == 1 :
            print('\r Step [{:5}/{} ({:.0f}%)] | Accuracy: {:.4f}%'.format(
                    batch_idx * len(data), 
                    len(test_loader.dataset),
                    100. * batch_idx / len(test_loader), 
                    (total_correct / total_target) * 100), end=' ')

 Step [ 4901/4927 (99%)] | Accuracy: 56.6911% CPU times: user 6.62 s, sys: 80 ms, total: 6.7 s
Wall time: 6.68 s
